<a href="https://colab.research.google.com/github/koopatroopa787/Knowledge_graph/blob/main/knowledge_graph(harry_potter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unstructured

In [ ]:
# prompt: install langchain
!pip install langchain
!pip install transformers

import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import json
from transformers import pipeline

In [4]:
# Create the input and output directories.
input_directory = Path("/content/input_directory")
output_directory = Path("/content/output_directory")

# Create the directories if they don't exist.
input_directory.mkdir(parents=True, exist_ok=True)
output_directory.mkdir(parents=True, exist_ok=True)

# Loader variable that loads the document The Best Games Of 2023.txt using DirectoryLoader
loader = DirectoryLoader(input_directory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


  0%|          | 0/1 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 1/1 [00:05<00:00,  5.98s/it]

Number of chunks =  34
Mr. Dursley always sat with his back to the window in his office on the ninth floor. If he hadn’t, he might have found it harder to concentrate on drills that morning. He didn’t see the owls swooping past in broad daylight, though people down in the street did; they pointed and gazed open-mouthed as owl after owl sped overhead. Most of them had never seen an owl even at nighttime. Mr. Dursley, however, had a perfectly normal, owl-free morning. He yelled at five different people. He made several important telephone Page | 4 Harry Potter and the Philosophers Stone - J.K. Rowling

calls and shouted a bit more. He was in a very good mood until lunchtime, when he thought he’d stretch his legs and walk across the road to buy himself a bun from the bakery.

He’d forgotten all about the people in cloaks until he passed a group of them next to the baker’s. He eyed them angrily as he passed. He didn’t know why, but they made him uneasy. This bunch were whispering excitedly

In [5]:
import uuid
import pandas as pd
import numpy as np


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


In [6]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(34, 3)


,text,source,chunk_id
0,"THE BOY WHO LIVED\n\nMr. and Mrs. Dursley, of ...",/content/input_directory/THE BOY WHO LIVED .txt,22da8a2992524eb7a79c6b4d3f209c18
1,This boy was another good reason for keeping t...,/content/input_directory/THE BOY WHO LIVED .txt,3d4b8a82827048ebb779415f8c65b8f3
2,the cat. It stared back. As Mr. Dursley drove ...,/content/input_directory/THE BOY WHO LIVED .txt,ec0d951392d047679a824bd6383445e8
3,Mr. Dursley always sat with his back to the wi...,/content/input_directory/THE BOY WHO LIVED .txt,aa329ec9274541b796d0498287930a2e
4,Mr. Dursley stopped dead. Fear flooded him. He...,/content/input_directory/THE BOY WHO LIVED .txt,36f9655fac4d4f1492c21d73fc1e04df


In [7]:
import json

def graphPrompt(input_text: str, metadata={}, model_pipeline=None):
    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "Your task is to extract the ontology of terms mentioned in the given context. "
        "Format your output as a list of json. Each element of the list contains a pair of terms "
        "and the relation between them, like the following: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input_text}``` \n\n output: "

    # Generate the response using the model pipeline
    response = model_pipeline(SYS_PROMPT + USER_PROMPT, max_length=1000, num_return_sequences=1)

    # Extract the "generated_text" from the response
    generated_text = response[0]['generated_text']

    try:
        # Attempt to parse the generated text as JSON
        result = json.loads(generated_text.split('output: ')[-1])
        # Merge the metadata with each item in the result
        result = [dict(item, **metadata) for item in result]
        print("Graph Prompt Result:", result)  # Display the result
    except json.JSONDecodeError as e:
        print("\n\nERROR ### Here is the buggy response: ", generated_text, "\n\n")
        result = None
    return result



In [8]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Define the model name and load the model pipeline
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1000)

# Define the output directory
output_directory = 'output_directory'
os.makedirs(output_directory, exist_ok=True)

def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list

def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

# Process the DataFrame and generate the graph
regenerate = True
if regenerate:
    concepts_list = df2Graph(df, model=model_pipeline)
    dfg1 = graph2Df(concepts_list)

    # Save the graph to a CSV file
    graph_csv_path = os.path.join(output_directory, "graph.csv")
    dfg1.to_csv(graph_csv_path, sep="|", index=False)

    # Optionally, save the original chunks to another CSV file
    chunks_csv_path = os.path.join(output_directory, "chunks.csv")
    df.to_csv(chunks_csv_path, sep="|", index=False)
else:
    dfg1 = pd.read_csv(os.path.join(output_directory, "graph.csv"), sep="|")

# Replace empty strings with NaN and drop rows with NaN values
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4  # Increasing the weight of the relation to 4

print(dfg1.shape)
dfg1.head()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Graph Prompt Result: [{'node_1': 'Dreams', 'node_2': 'Reality', 'edge': "Dreams are not a part of reality, Harry wished he hadn't mentioned his dream to the Dursleys as they dislike anything that doesn't conform to the expected behavior.", 'chunk_id': '46a1d0d5311245d698074b7e70a098e9'}, {'node_1': 'Dudley and Piers', 'node_2': 'Harry', 'edge': 'Dudley and Piers are the abusers of Harry, they dislike Harry and often hit him.', 'chunk_id': '46a1d0d5311245d698074b7e70a098e9'}, {'node_1': 'Zoo', 'node_2': 'Family outing', 'edge': 'Zoo is a place where families go for entertainment and recreation.', 'chunk_id': '46a1d0d5311245d698074b7e70a098e9'}, {'node_1': 'Uncle Vernon', 'node_2': 'Dudley and Piers', 'edge': 'Uncle Vernon is the guardian of Dudley and Piers.', 'chunk_id': '46a1d0d5311245d698074b7e70a098e9'}, {'node_1': 'Harry', 'node_2': 'Animals', 'edge': 'Harry observes animals at the zoo.', 'chunk_id': '46a1d0d5311245d698074b7e70a098e9'}, {'node_1': 'Dudley', 'node_2': 'Gorilla', 'ed

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




ERROR ### Here is the buggy response:  You are a network graph maker who extracts terms and their relations from a given context. Your task is to extract the ontology of terms mentioned in the given context. Format your output as a list of json. Each element of the list contains a pair of terms and the relation between them, like the following: 
[
   {
       "node_1": "A concept from extracted ontology",
       "node_2": "A related concept from extracted ontology",
       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"
   }, {...}
]context: ```Harry felt, afterward, that he should have known it was all too good to last.

After lunch they went to the reptile house. It was cool and dark in there, with lit windows all along the walls. Behind the glass, all sorts of lizards and snakes were crawling and slithering over bits of wood and stone. Dudley and Piers wanted to see huge, poisonous cobras and thick, man-crushing pythons. Dudley quickly fo

,node_1,node_2,edge,chunk_id,count
0,mr. dursley,owls,Mr. Dursley didn’t see the owls swooping past ...,aa329ec9274541b796d0498287930a2e,4
1,mr. dursley,people down in the street,People down in the street saw the owls swoopin...,aa329ec9274541b796d0498287930a2e,4
2,mr. dursley,group of people in cloaks,Mr. Dursley eyed the group of people in cloaks...,aa329ec9274541b796d0498287930a2e,4
3,group of people in cloaks,the potters,The group of people in cloaks were whispering ...,aa329ec9274541b796d0498287930a2e,4
4,group of people in cloaks,harry,The group of people in cloaks were whispering ...,aa329ec9274541b796d0498287930a2e,4


In [9]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
406,voldemort,professor mcgonagall,"bf06cf1555d24d7980a00a10396f8a75,bf06cf1555d24...",6,contextual proximity
408,you-know-who,mr. dursley,"4e0363abf24546f798d691f4ddc0d438,4e0363abf2454...",2,contextual proximity
410,you-know-who,stranger,"4e0363abf24546f798d691f4ddc0d438,4e0363abf2454...",3,contextual proximity
414,zoo,dudley and piers,"46a1d0d5311245d698074b7e70a098e9,46a1d0d531124...",2,contextual proximity
417,zoo,harry,"46a1d0d5311245d698074b7e70a098e9,46a1d0d531124...",2,contextual proximity


In [10]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,animals,dudley and piers,"46a1d0d5311245d698074b7e70a098e9,46a1d0d531124...",contextual proximity,2
1,animals,harry,"46a1d0d5311245d698074b7e70a098e9,46a1d0d531124...",contextual proximity,2
2,annoying,harry,"133601aa516b43de9ae1a7191c70dabd,133601aa516b4...",contextual proximity,3
3,annoying,snake,"133601aa516b43de9ae1a7191c70dabd,133601aa516b4...",contextual proximity,9
4,anyone,harry,"133601aa516b43de9ae1a7191c70dabd,133601aa516b4...",contextual proximity,3
...,...,...,...,...,...
244,you-know-who,mr. dursley,"4e0363abf24546f798d691f4ddc0d438,4e0363abf2454...",contextual proximity,2
245,you-know-who,stranger,"4e0363abf24546f798d691f4ddc0d438,4e0363abf2454...",contextual proximity,3
246,zoo,dudley and piers,"46a1d0d5311245d698074b7e70a098e9,46a1d0d531124...",contextual proximity,2
247,zoo,family outing,46a1d0d5311245d698074b7e70a098e9,Zoo is a place where families go for entertain...,4


In [11]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(58,)

In [12]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [13]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  3
[['animals', 'annoying', 'anyone', 'aunt petunia', 'beady eyes', 'ceiling', 'dreams', 'dudley', 'dudley and piers', "dudley's gang", 'dursleys', 'family outing', 'glass', 'gorilla', 'harry', 'headmistress', 'marge', 'motorcycles', 'mrs. figg', 'reality', 'school kitchens', 'snake', 'uncle vernon', 'uncle vernon and dudley', 'zoo'], ['boy who lived', 'bringing harry potter', 'bundle of blankets', 'dumbledore', 'glasses', 'hagrid', 'harry potter', "harry potter's aunt and uncle", 'lily and james potter', 'motorcycle', 'people', 'privet drive', 'professor mcgonagall', 'sirius black', 'tabby cat', 'vanquished', 'voldemort'], ['cat', 'concentration', 'fear', 'group of people in cloaks', 'his nephew', 'his sister', 'mr. dursley', 'muggle', 'owls', 'people down in the street', 'people in cloaks', 'stranger', 'the cat', 'the potters', 'upset', 'you-know-who']]


In [14]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors


,node,color,group
0,animals,#57db5f,1
1,annoying,#57db5f,1
2,anyone,#57db5f,1
3,aunt petunia,#57db5f,1
4,beady eyes,#57db5f,1
5,ceiling,#57db5f,1
6,dreams,#57db5f,1
7,dudley,#57db5f,1
8,dudley and piers,#57db5f,1
9,dudley's gang,#57db5f,1


In [15]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [ ]:
!pip install pyvis


In [16]:
import os

# Define the directory path
directory_path = "./docs"

# Check if the directory exists
if not os.path.exists(directory_path):
    # If it does not exist, create the directory
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' was created.")
else:
    print(f"Directory '{directory_path}' already exists.")


Directory './docs' was created.


In [18]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html


In [ ]:
import networkx as nx


In [ ]:
cdn_resources="remote"
g = Network(height=800,width=800,notebook=True)
g.toggle_hide_edges_on_drag(False)
g.barnes_hut()
g.from_nx(nx.davis_southern_women_graph())
g.show('/content/docs/index.html')